In [4]:
from google import genai
import os
import re
import httpx
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv("GEMINI_API_KEY")

client = genai.Client(api_key=api_key)

In [5]:
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "systems", "content": system})
    
    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result
    
    def execute(self):
        response = client.models.generate_content(
            model="gemini-2.0-flash",
            contents=[self.messages],
            config=genai.types.GenerateContentConfig(
                max_output_tokens=500,
                temperature=0
            )
        )
        return response.text

In [6]:
prompt = """
You operate in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop, you output an Answer.

Use Thought to explain your reasoning about the travel-related question.

Use Action to invoke one of the available tools — then return PAUSE.

Observation will be the result of that action.

Your available actions are:
flight_duration:
e.g. flight_duration: New York to Tokyo
Returns the average flight time between two cities.

currency_conversion:
e.g. currency_conversion: 100 USD to EUR
Returns the equivalent value in the target currency based on current rates.

visa_requirement:
e.g. visa_requirement: India to Germany
Returns visa requirement information for a citizen of the first country traveling to the second.

Question: How long is the flight from London to Tokyo?
Thought: I should use the flight_duration action to find the average time for this route.
Action: flight_duration: London to Dubai
PAUSE

You will then be called again with:

Observation: The average flight from London to Tokyo takes about 7 hours.

You then respond:

Answer: The average flight time from London to Tokyo is around 21 hours
""".strip()

In [7]:
def flight_duration(route):
    durations = {
        "New York to Tokyo": "14 hours",
        "London to Dubai": "7 hours",
        "Sydney to Los Angeles": "13.5 hours",
        "London to Tokyo": "21 hours"
        }
    return durations.get(route, "Flight duration data not available")

def currency_conversion(conversion):
    # Very basic mock logic
    try:
        amount, from_to = conversion.split(" ", 1)
        amount = float(amount)
        if "USD to EUR" in from_to:
            return f"{amount * 0.91:.2f} EUR"
        elif "EUR to USD" in from_to:
            return f"{amount * 1.10:.2f} USD"
        elif "USD to JPY" in from_to:
            return f"{amount * 150.0:.2f} JPY"
        else:
            return "Conversion rate not available"
    except:
        return "Invalid conversion format"

def visa_requirement(route):
    requirements = {
        "India to Germany": "Yes, Indian citizens need a visa to travel to Germany.",
        "USA to Japan": "No visa required for stays up to 90 days.",
        "UK to Australia": "Yes, a visa is required for UK citizens visiting Australia.",
        "UK to Japan": "No visa required for UK citizens visiting Japan for tourism."
        }
    return requirements.get(route, "Visa requirement data not available")

known_actions = {
    "flight_duration": flight_duration,
    "currency_conversion": currency_conversion,
    "visa_requirement": visa_requirement
}

In [11]:
agentbot = Agent(prompt)
print(type(agentbot.system))


<class 'str'>


In [9]:
result = agentbot("How long is the flight from London to Tokyo?")
print(result)

ValidationError: 21 validation errors for _GenerateContentParameters
contents.list[union[Content,list[union[File,Part,str]],File,Part,str]].0.Content
  Input should be a valid dictionary or object to extract fields from [type=model_attributes_type, input_value=[{'role': 'systems', 'con...from London to Tokyo?'}], input_type=list]
    For further information visit https://errors.pydantic.dev/2.11/v/model_attributes_type
contents.list[union[Content,list[union[File,Part,str]],File,Part,str]].0.list[union[File,Part,str]].0.File.role
  Extra inputs are not permitted [type=extra_forbidden, input_value='systems', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/extra_forbidden
contents.list[union[Content,list[union[File,Part,str]],File,Part,str]].0.list[union[File,Part,str]].0.File.content
  Extra inputs are not permitted [type=extra_forbidden, input_value='You operate in a loop of...okyo is around 21 hours', input_type=str]
    For further information visi